In [3]:
import functools
import nest_asyncio
import os
import traceback
import pandas as pd
import telebot
from icecream import ic
from IPython import get_ipython
from IPython.utils.capture import capture_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain import hub
import importlib
import json

# Apply nest_asyncio
nest_asyncio.apply()

# Load configuration from config.json
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"
TELEGRAMM_API_KEY = config["TELEGRAM_BOT_TOKEN"]

# Import and reload the gsheet_handler module
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Extract data from Google Sheets
example_dialog = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0', 0)
example_dialog.to_csv('example_dialog.csv', index=False)

support = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1C3m50L3iUWhkkrp5CWEmksVRAkA8y4hX6m2tYgS8-4I/edit?gid=1891107571#gid=1891107571', 0)
support.to_csv('support.csv', index=False)

# New Google Sheets link for "помещения и сооружения"
facilities = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/19oNWcJstB1fJRqJAbi1YKygUlMqs91fF8-Jw-OTvPXo/edit?gid=362855495#gid=362855495', 0)
facilities.to_csv('facilities.csv', index=False)

from langchain_experimental.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.agents import AgentType

# Create a CSV agent using LangChain and OpenAI with three CSV files
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4o"),
    ["example_dialog.csv", "support.csv", "facilities.csv"],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,  # Opt-in to use the functionality
)

# Run some queries using the agent
print(agent.run("how many rows are there in each table?"))
print(agent.run("how many rows in the support table?"))
print(agent.run("how many rows in the example_dialog table?"))
print(agent.run("how many rows in the facilities table?"))


no docx


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'rows_df1 = len(df1)\nrows_df2 = len(df2)\nrows_df3 = len(df3)\nrows_df1, rows_df2, rows_df3'}`


(18, 120, 196)The number of rows in each table is as follows:
- **df1**: 18 rows
- **df2**: 120 rows
- **df3**: 196 rows

> Finished chain.
The number of rows in each table is as follows:
- **df1**: 18 rows
- **df2**: 120 rows
- **df3**: 196 rows


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\ndf2 = pd.read_csv('support_table.csv')\nlen(df2)"}`


FileNotFoundError: [Errno 2] No such file or directory: 'support_table.csv'
Invoking: `python_repl_ast` with `{'query': 'len(df2)'}`


120The support table (df2) contains 120 rows.

> Finished chain.
The support table (df2) contains 120 rows.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf3 = pd.read_csv('example_dialog.csv')\nlen(df3)